<a href="https://colab.research.google.com/github/solak647/ATML19/blob/master/model_thomas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import numpy as np
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [0]:
from torchvision.datasets import ImageFolder
from torchvision.transforms import Resize, ToTensor, Normalize, Compose, Grayscale, ColorJitter
from torch.utils.data import DataLoader
batch_size = 20
data_dir = '/content/drive/My Drive/ATML/data/'
root_dir = data_dir + 'train'

target_size = (100,100)
transforms = Compose([
                    Grayscale(num_output_channels=1),
                    ColorJitter(brightness=0.2,contrast=0.2),
                   # Resize(target_size), # Resizes image
                    ToTensor(),           # Converts to Tensor, scales to [0, 1] float (from [0, 255] int)
                    Normalize((0.5,), (0.5,)), # scales to [-1.0, 1.0]
                    ])

train_dataset = ImageFolder(root_dir, transform=transforms)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=32)

In [0]:
# Same for validation dataset
val_root_dir = data_dir + 'val'
val_dataset = ImageFolder(val_root_dir, transform=transforms)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True, num_workers=4)

In [0]:
# Same for test dataset
test_root_dir = data_dir + 'test'
test_dataset = ImageFolder(test_root_dir, transform=transforms)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True, num_workers=4)

In [0]:
print(len(train_dataset))

4194


In [0]:
import torch.nn as nn
    
class ConvNet(nn.Module):
    
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv = nn.Sequential(
          # input: 1x216x216
          nn.Conv2d(1, 64, 5),
          # output: 64x212x212
          nn.LeakyReLU(0.2),
          nn.MaxPool2d(2, 2),
          # output: 64x106x106
          nn.Conv2d(64, 64, 5),
          # output: 64x102x102
          nn.LeakyReLU(0.2),
          nn.MaxPool2d(2, 2),
          # output: 64x51x51
          nn.Conv2d(64, 64, 4),
          # output: 64x48x48
          nn.LeakyReLU(0.2),
          nn.MaxPool2d(2, 2),
          # output: 64x24x24
          nn.Conv2d(64, 64, 5),
          # output: 64x20*20
          nn.LeakyReLU(0.2),
          nn.MaxPool2d(2, 2),
          # output: 64x10*10
        )
        self.fc = nn.Sequential(
          nn.Linear(64*10*10,364),
          nn.LeakyReLU(0.2),
          nn.Dropout(0.5),
          nn.Linear(364,192),
          nn.LeakyReLU(0.2),
          nn.Dropout(0.5),
          nn.Linear(192,10)
        )
    
    def forward(self, input):
        output = self.conv(input)
        output = output.view(output.size(0), 64*10*10)
        output = self.fc(output)
        return output

In [0]:
# ADDING EARLY STOPPING
import numpy as np
import torch
from copy import deepcopy
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

def train(model, train_loader, optimizer, loss_fn, print_every=100):
    '''
    Trains the model for one epoch
    '''
    model.train()
    losses = []
    n_correct = 0
    for iteration, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        output = model(images)
        optimizer.zero_grad()
        loss = loss_fn(output, labels)
        loss.backward()
        optimizer.step()
#         if iteration % print_every == 0:
#             print('Training iteration {}: loss {:.4f}'.format(iteration, loss.item()))
        losses.append(loss.item())
        n_correct += torch.sum(output.argmax(1) == labels).item()
    accuracy = 100.0 * n_correct / len(train_loader.dataset)
    return np.mean(np.array(losses)), accuracy
            
def test(model, test_loader, loss_fn):
    '''
    Tests the model on data from test_loader
    '''
    model.eval()
    test_loss = 0
    n_correct = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)
            output = model(images)
            loss = loss_fn(output, labels)
            test_loss += loss.item()
            n_correct += torch.sum(output.argmax(1) == labels).item()

    average_loss = test_loss / len(test_loader)
    accuracy = 100.0 * n_correct / len(test_loader.dataset)
#     print('Test average loss: {:.4f}, accuracy: {:.3f}'.format(average_loss, accuracy))
    return average_loss, accuracy


def fit(train_dataloader, val_dataloader, model, optimizer, loss_fn, n_epochs, scheduler=None):
    train_losses, train_accuracies = [], []
    val_losses, val_accuracies = [], []
    best_val_loss = np.inf
    val_accuracy_best = 0
    best_model = None
    patience = 5 # if no improvement after 5 epochs, stop training
    counter = 0
    for epoch in range(n_epochs):
        train_loss, train_accuracy = train(model, train_dataloader, optimizer, loss_fn)
        val_loss, val_accuracy = test(model, val_dataloader, loss_fn)
        train_losses.append(train_loss)
        train_accuracies.append(train_accuracy)
        val_losses.append(val_loss)
        val_accuracies.append(val_accuracy)
        if scheduler:
            scheduler.step() # argument only needed for ReduceLROnPlateau
        print('Epoch {}/{}: train_loss: {:.4f}, train_accuracy: {:.4f}, val_loss: {:.4f}, val_accuracy: {:.4f}'.format(epoch+1, n_epochs,
                                                                                                          train_losses[-1],
                                                                                                          train_accuracies[-1],
                                                                                                          val_losses[-1],
                                                                                                          val_accuracies[-1]))
        ### Early stopping code
        if val_accuracy > val_accuracy_best:
            best_val_loss = val_loss
            val_accuracy_best = val_accuracy
            best_model = deepcopy(model)
            counter = 0
        else:
            counter += 1
        if counter == patience and False:
            print('No improvement for {} epochs; training stopped.'.format(patience))
            break
    
    return best_val_loss, val_accuracy_best

In [0]:
model_conv = ConvNet()
model_conv = model_conv.to(device)
learning_rate = 0.001
optimizer = torch.optim.Adam(model_conv.parameters(), lr=learning_rate, weight_decay=0.004)
n_epochs = 25
loss_fn = nn.CrossEntropyLoss()

In [0]:
val_loss, val_accuracy = fit(train_dataloader, val_dataloader, model_conv, optimizer, loss_fn, n_epochs)

Epoch 1/25: train_loss: 2.4935, train_accuracy: 10.0000, val_loss: 2.3419, val_accuracy: 10.0000
Epoch 2/25: train_loss: 2.7547, train_accuracy: 8.2222, val_loss: 3.0683, val_accuracy: 10.6667
Epoch 3/25: train_loss: 2.1891, train_accuracy: 27.8889, val_loss: 9.3977, val_accuracy: 10.0000
Epoch 4/25: train_loss: 2.6505, train_accuracy: 28.6667, val_loss: 5.1078, val_accuracy: 10.0000
Epoch 5/25: train_loss: 2.0266, train_accuracy: 38.2778, val_loss: 20.2907, val_accuracy: 10.0000
Epoch 6/25: train_loss: 7.4725, train_accuracy: 18.2222, val_loss: 3.6653, val_accuracy: 11.3333
Epoch 7/25: train_loss: 2.9744, train_accuracy: 24.6667, val_loss: 2.3443, val_accuracy: 11.0000
Epoch 8/25: train_loss: 3.0941, train_accuracy: 9.8333, val_loss: 6.7081, val_accuracy: 10.0000
Epoch 9/25: train_loss: 2.6864, train_accuracy: 17.8333, val_loss: 31.3317, val_accuracy: 10.0000
Epoch 10/25: train_loss: 3.4473, train_accuracy: 9.5000, val_loss: 3.3121, val_accuracy: 10.0000
Epoch 11/25: train_loss: 3.324

In [0]:
import torch.nn as nn
    
class Conv1DNet(nn.Module):
    
    def __init__(self):
        super(Conv1DNet, self).__init__()
        self.conv = nn.Sequential(
          # input: 1x216x216
          nn.Conv2d(1, 64, (5,1)),
          # output: 64x212x216
          nn.BatchNorm2d(64,momentum=0.9),
          nn.ReLU(),
          nn.MaxPool2d((2,1), (2,1)),
          nn.Dropout(0.5),
          # output: 64x106x216
          nn.Conv2d(64, 64, (5,1)),
          # output: 64x102x216
          nn.BatchNorm2d(64,momentum=0.9),
          nn.ReLU(),
          nn.MaxPool2d((2,1), (2,1)),
          nn.Dropout(0.5),
          # output: 64x51x216
          nn.Conv2d(64, 64, (4,1)),
          # output: 64x48x216
          nn.BatchNorm2d(64,momentum=0.9),
          nn.ReLU(),
          nn.MaxPool2d((2,1), (2,1)),
          nn.Dropout(0.5),
          # output: 64x24x216
          nn.Conv2d(64, 64, (5,1)),
          # output: 64x20x216
          nn.BatchNorm2d(64,momentum=0.9),
          nn.ReLU(),
          nn.MaxPool2d((2,1), (2,1)),
          nn.Dropout(0.5)
          # output: 64x10x216
        )
        self.fc = nn.Sequential(
          nn.Linear(64*10*216,364),
          nn.LeakyReLU(0.2),
          nn.Dropout(0.5),
          nn.Linear(364,192),
          nn.LeakyReLU(0.2),
          nn.Dropout(0.5),
          nn.Linear(192,10)
        )
    
    def forward(self, input):
        output = self.conv(input)
        output = output.view(output.size(0), 64*10*216)
        output = self.fc(output)
        return output

In [0]:
model_conv = Conv1DNet()
model_conv = model_conv.to(device)
learning_rate = 0.001
optimizer = torch.optim.Adam(model_conv.parameters(), lr=learning_rate)
n_epochs = 50
loss_fn = nn.CrossEntropyLoss()

In [0]:
val_loss, val_accuracy = fit(train_dataloader, val_dataloader, model_conv, optimizer, loss_fn, n_epochs)

Epoch 1/50: train_loss: 39.1402, train_accuracy: 9.2222, val_loss: 21.6251, val_accuracy: 10.0000
Epoch 2/50: train_loss: 20.2774, train_accuracy: 2.1667, val_loss: 8.2255, val_accuracy: 9.6667
Epoch 3/50: train_loss: 10.4945, train_accuracy: 1.8333, val_loss: 2.7083, val_accuracy: 13.6667
Epoch 4/50: train_loss: 4.7679, train_accuracy: 3.6667, val_loss: 2.3164, val_accuracy: 18.3333
Epoch 5/50: train_loss: 3.6239, train_accuracy: 5.0556, val_loss: 2.3173, val_accuracy: 13.6667
Epoch 6/50: train_loss: 3.3256, train_accuracy: 5.1667, val_loss: 2.2237, val_accuracy: 23.0000
Epoch 7/50: train_loss: 2.9765, train_accuracy: 6.5556, val_loss: 2.2480, val_accuracy: 15.0000
Epoch 8/50: train_loss: 2.7924, train_accuracy: 7.3333, val_loss: 2.2407, val_accuracy: 19.0000
Epoch 9/50: train_loss: 2.7316, train_accuracy: 7.7222, val_loss: 2.2572, val_accuracy: 19.6667
Epoch 10/50: train_loss: 2.5959, train_accuracy: 9.3889, val_loss: 2.2552, val_accuracy: 24.0000
Epoch 11/50: train_loss: 2.5691, tra

In [0]:
import torch.nn as nn
    
class Conv1DNet2(nn.Module):
    
    def __init__(self):
        super(Conv1DNet2, self).__init__()
        self.conv = nn.Sequential(
          # input: 1x216x216
          nn.Conv2d(1, 128, (5,1)),
          # output: 128x212x216
          nn.BatchNorm2d(128,momentum=0.9),
          nn.LeakyReLU(0.2),
          nn.MaxPool2d(2),
          nn.Dropout(0.5),
          # output: 128x106x108
          nn.Conv2d(128, 64, (5,1)),
          # output: 64x102x108
          nn.BatchNorm2d(64,momentum=0.9),
          nn.LeakyReLU(0.2),
          nn.MaxPool2d(2),
          nn.Dropout(0.5),
          # output: 64x51x54
          nn.Conv2d(64, 64, (4,1)),
          # output: 64x48x54
          nn.BatchNorm2d(64,momentum=0.9),
          nn.LeakyReLU(0.2),
          nn.MaxPool2d(2),
          nn.Dropout(0.5),
          # output: 64x24x27
          nn.Conv2d(64, 64, (5,1)),
          # output: 64x20x27
          nn.BatchNorm2d(64,momentum=0.9),
          nn.LeakyReLU(0.2),
          nn.MaxPool2d((2,1), (2,1)),
          nn.Dropout(0.5)
          # output: 64x10x27
        )
        self.fc = nn.Sequential(
          nn.Linear(64*10*27,364),
          nn.LeakyReLU(0.2),
          nn.Dropout(0.5),
          nn.Linear(364,192),
          nn.LeakyReLU(0.2),
          nn.Dropout(0.5),
          nn.Linear(192,10)
        )
    
    def forward(self, input):
        output = self.conv(input)
        output = output.view(output.size(0), 64*10*27)
        output = self.fc(output)
        return output

In [0]:
model_conv = Conv1DNet2()
model_conv = model_conv.to(device)
learning_rate = 0.001
optimizer = torch.optim.Adam(model_conv.parameters(), lr=learning_rate)
n_epochs = 50
loss_fn = nn.CrossEntropyLoss()

In [0]:
val_loss, val_accuracy = fit(train_dataloader, val_dataloader, model_conv, optimizer, loss_fn, n_epochs)

Epoch 1/50: train_loss: 2.1581, train_accuracy: 22.8422, val_loss: 1.9567, val_accuracy: 29.0000
Epoch 2/50: train_loss: 1.7466, train_accuracy: 36.5761, val_loss: 1.6314, val_accuracy: 41.2857
Epoch 3/50: train_loss: 1.4952, train_accuracy: 47.0911, val_loss: 1.2620, val_accuracy: 54.8571
Epoch 4/50: train_loss: 1.3055, train_accuracy: 53.3143, val_loss: 1.1972, val_accuracy: 59.0000
Epoch 5/50: train_loss: 1.2344, train_accuracy: 55.0548, val_loss: 1.1767, val_accuracy: 60.0000
Epoch 6/50: train_loss: 1.1229, train_accuracy: 59.9189, val_loss: 1.1844, val_accuracy: 61.1429
Epoch 7/50: train_loss: 1.0484, train_accuracy: 63.3286, val_loss: 1.3152, val_accuracy: 56.5714
Epoch 8/50: train_loss: 0.9646, train_accuracy: 66.5474, val_loss: 1.0257, val_accuracy: 64.0000
Epoch 9/50: train_loss: 0.8899, train_accuracy: 69.3848, val_loss: 1.4360, val_accuracy: 59.2857
Epoch 10/50: train_loss: 0.8420, train_accuracy: 70.4340, val_loss: 1.0520, val_accuracy: 67.5714
Epoch 11/50: train_loss: 0.76